# アプリケーションの実行

BDC には、Python / R / SSIS / MLEAP をアプリケーションとして展開することができる


## 1. テンプレートからアプリケーションをデプロイ
**1. BDC に接続**  
1. 接続先の IP を確認
```
kubectl get service -n mssql-cluster | findstr "30777"
```
**2. アプリケーションテンプレートの作成**  
1. C:\Users\decodeadmin\Desktop\Demo\04.Integrated Data Access\04.Application on BDC\bdc-app を開く
1. アプリケーションテンプレートとして　Python / R / SSIS / MLeap が用意されている  
![BDC Application Template](https://raw.githubusercontent.com/MasayukiOzawa/decode-2019-demo/master/Images/04.Integrated%20Data%20Access/04.Application%20on%20BDC/Application%20Template.png)


**3. サンプルアプリケーションのデプロイ**
1. spec.yaml をデプロイ
1. BDC にアプリが展開されたことを確認  
![BDC App](https://raw.githubusercontent.com/MasayukiOzawa/decode-2019-demo/master/Images/04.Integrated%20Data%20Access/04.Application%20on%20BDC/BDC%20App%20Explorer.png)
1. [Swagger](https://editor.swagger.io/#) の利用  
![Swagger](https://github.com/MasayukiOzawa/decode-2019-demo/raw/master/Images/04.Integrated%20Data%20Access/04.Application%20on%20BDC/Swagger.png)


## 2. アプリケーションの実行
1. run.yaml を使用してアプリケーションを実行  
![Run App](https://github.com/MasayukiOzawa/decode-2019-demo/raw/master/Images/04.Integrated%20Data%20Access/04.Application%20on%20BDC/Run%20App.png)
1. コマンドから実行
```
mssqlctl login -n mssql-cluster -u $ENV:MGMTPROXY_LOGIN
mssqlctl app run -n add-app -v v1 --inputs x=10 y=120
```
3. コードから実行
```
& "C:\Users\decodeadmin\Desktop\Demo\04.Integrated Data Access\04.Application on BDC\02.BDC Application.ps1"
```
  
```
$body = @{
    x =100
    y= 20
}


$temp = (kubectl get services --namespace mssql-cluster | Select-String "30777") -split " "
$ret = @()
foreach($local:v in $temp){
    if ($v -ne ""){
        $ret += $v
    }
}

Add-type @"
    using System.Net;
    using System.Security.Cryptography.X509Certificates;
    public class TrustAllCertsPolicy : ICertificatePolicy {
        public bool CheckValidationResult(
            ServicePoint srvPoint, X509Certificate certificate,
            WebRequest request, int certificateProblem) {
            return true;
        }
    }
"@
[System.Net.ServicePointManager]::CertificatePolicy = New-Object TrustAllCertsPolicy
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Ssl3, [Net.SecurityProtocolType]::Tls, [Net.SecurityProtocolType]::Tls11, [Net.SecurityProtocolType]::Tls12

$User = $env:MGMTPROXY_LOGIN
$Password = $env:MGMTPROXY_LOGIN_PASSWORD
$cred = New-Object System.Management.Automation.PSCredential($User, (ConvertTo-SecureString -String $Password -AsPlainText -Force))

$token = (Invoke-WebRequest -Uri "https://$($ret[3]):30777/api/token" -Method Post -Credential $cred).Content | ConvertFrom-Json

$header = @{
  "Authorization" = ("Bearer {0}" -f $token.access_token);
  "content-type" = 'application/json'
}


$temp = (kubectl get services --namespace mssql-cluster | Select-String "30778") -split " "
$app = @()
foreach($local:v in $temp){
    if ($v -ne ""){
        $app += $v
    }
}


$ret = Invoke-WebRequest -Uri "https://$($app[3]):30778/api/app/add-app/v1/run" -Method Post -Headers $header -Body ($body | ConvertTo-Json)

$ret.Content | ConvertFrom-Json
```
